# Agentic Maintenance Planner
### OpenEnv + Unsloth + TRL PPO (Colab-first)


In [ ]:
!pip install -r ../requirements.txt


In [ ]:
from unsloth import FastLanguageModel
from trl import PPOTrainer, PPOConfig
from transformers import AutoTokenizer
from environment.openenv_maintenance_env import MaintenancePlannerEnv


In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name='meta-llama/Llama-3-8B-Instruct',
    load_in_4bit=True,
    max_seq_length=2048,
)


In [ ]:
ppo_config = PPOConfig(batch_size=1, learning_rate=1e-5)
trainer = PPOTrainer(config=ppo_config, model=model, tokenizer=tokenizer)


In [ ]:
env = MaintenancePlannerEnv()
obs, _ = env.reset()

for step in range(10):
    prompt = f'Asset health is {obs['asset_health']}. What should I do?'
    inputs = tokenizer(prompt, return_tensors='pt')
    action = 0 if obs['asset_health'] < 70 else 1
    obs, reward, done, _, _ = env.step(action)
    trainer.step([prompt], [inputs['input_ids']], [reward])
